Nessie Demo
===========
This demo showcases how to use Nessie python API along with Spark

Initialize Pyspark + Nessie environment
----------------------------------------------

In [ ]:
import findspark
from pyspark.sql import *
from pyspark import SparkConf, SparkContext
from py4j.java_gateway import java_import
findspark.init()

spark = SparkSession.builder \
                    .config("spark.jars", "../../clients/deltalake-spark3/target/nessie-deltalake-spark3-1.0-SNAPSHOT.jar") \
                    .config("spark.sql.execution.arrow.enabled", "true") \
                    .config("spark.hadoop.fs.defaultFS", 'file:///home/ryan/workspace/tick_data/spark_warehouse') \
                    .config("spark.hadoop.nessie.url", "http://localhost:19120/api/v1") \
                    .config("spark.hadoop.nessie.ref", "main") \
                    .config("spark.sql.catalog.nessie", "com.dremio.nessie.iceberg.spark.NessieDeltaSparkCatalog") \
                    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
                    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
                    .config("spark.delta.logFileHandler.class", "com.dremio.nessie.deltalake.NessieLogFileMetaParser") \
                    .config("spark.delta.logStore.class", "com.dremio.nessie.deltalake.NessieLogStore") \
                    .getOrCreate()
sc = spark.sparkContext
jvm = sc._gateway.jvm

java_import(jvm, "org.apache.spark.sql.delta.DeltaLog")
java_import(jvm, "io.delta.tables.DeltaTable")

Set up nessie branches
----------------------------

- Branch `main` already exists
- Create branch `dev`
- List all branches (pipe JSON result into jq)

In [ ]:
!nessie_client create-branch dev

In [ ]:
!nessie_client list-references | jq .

Create tables under dev branch
-------------------------------------

Creating two tables under the `dev` branch:
- region
- nation

It is not yet possible to create table using pyspark and iceberg, so Java code is used instead

In [ ]:
hadoop_conf = sc._jsc.hadoopConfiguration()
hadoop_conf.set("nessie.ref", "dev")

region_df = spark.read.load("data/region.parquet")
region_df.write.format("delta").save("spark_warehouse/testing/region")

nation_df = spark.read.load("data/nation.parquet")
nation_df.write.format("delta").save("spark_warehouse/testing/nation")


Check generated tables
----------------------------
   
Check tables generated under the dev branch (and that the main branch does not have any tables)

In [ ]:
!nessie_client list-tables main

In [ ]:
!nessie_client list-tables dev

In [ ]:
!nessie_client list-references | jq .

Dev promotion
-------------

Promote dev branch promotion to main

In [ ]:
!nessie_client assign-branch main dev

In [ ]:
!nessie_client list-tables main

In [ ]:
!nessie_client list-references | jq .

Create `etl` branch
----------------------

- Create a branch `etl` out of `main`
- add data to nation
- alter region
- create table city
- query the tables in `etl`
- query the tables in `main`
- promote `etl` branch to `main`

In [ ]:
!nessie_client create-branch etl -r `nessie_client show-reference main | jq .hash | sed 's/"//g'`

In [ ]:
hadoop_conf.set("nessie.ref", "etl")
Nation = Row("N_NATIONKEY", "N_NAME", "N_REGIONKEY", "N_COMMENT")
new_nations = spark.createDataFrame([
    Nation(25, "SYLDAVIA", 3, "King Ottokar's Sceptre"),
    Nation(26, "SAN THEODOROS", 1, "The Picaros")])
new_nations.write.option('hadoop.nessie.ref', 'etl').format("delta").mode("append").save("testing.nation")

In [ ]:
# changing the default branch
hadoop_conf.set('nessie.ref', 'etl')

In [ ]:
# Creating city table
sc.getConf().set("spark.hadoop.nessie.ref", "etl")
spark.sql("CREATE TABLE city (C_CITYKEY BIGINT, C_NAME STRING, N_NATIONKEY BIGINT, C_COMMNT STRING) USING delta PARTITIONED BY (N_NATIONKEY) LOCATION 'spark_warehouse/testing/city'")

In [ ]:
!nessie_client list-tables main

In [ ]:
!nessie_client list-tables etl

In [ ]:
!nessie_client list-references | jq .

In [ ]:
!nessie_client assign-branch main etl

In [ ]:
!nessie_client list-references | jq .

Create `experiment` branch
--------------------------------

- create `experiment` branch from `main`
- drop `nation` table
- add data to `region` table
- compare `experiment` and `main` tables

In [ ]:
!nessie_client create-branch experiment -r `nessie_client show-reference main | jq .hash | sed 's/"//g'`

In [ ]:
# changing the default branch
hadoop_conf.set('nessie.ref', 'experiment')


jvm.DeltaLog.clearCache()
deltaTable = jvm.DeltaTable.forPath("spark_warehouse/testing/nation")

In [ ]:
spark.sql("set spark.hadoop.nessie.ref=experiment")
spark.sql('INSERT INTO TABLE nessie.testing.region VALUES (5, "AUSTRALIA", "Let\'s hop there", "AUS")')
spark.sql('INSERT INTO TABLE nessie.testing.region VALUES (6, "ANTARTICA", "It\'s cold", "ANT")')

In [ ]:
!nessie_client list-tables experiment

In [ ]:
spark.sql("select * from nessie.testing.region").show()

In [ ]:
spark.sql("select * from nessie.testing.`region@main`").show()